## Happiness Metrics for Global Life Expectancy Predictive Analysis
Nicole Chang, Sourish Guntipally, Aaron Park, Brandon To

Data used for this project is from Kaggle datasets of the [World Happiness Report](https://www.kaggle.com/unsdsn/world-happiness?select=2019.csv) and [Human Life Expectancy Around the World](https://www.kaggle.com/deepcontractor/human-life-expectancy-around-the-world).

The years we are analyzing are from 2015 - 2019.

In [5]:
import scipy as sp, numpy as np, pandas as pd
from sklearn.cross_decomposition import PLSRegression
import matplotlib.pyplot as plt

In [24]:
#World Happiness Report (WHR)
#Human Life Expectancy Around the World (HLE)

#HLE
dataHLE = pd.read_csv('Human_life_Expectancy.csv')

#2015 data
dataWHR2015 = pd.read_csv('2015.csv') #WHR
dataHLE2015 = dataHLE[['2015']].to_numpy() #HLE
#figure out how to convert datatype to integers so we can actually work with the numbers

#2016 data
dataWHR2016 = pd.read_csv('2016.csv') #WHR

#2017 data
dataWHR2017 = pd.read_csv('2017.csv') #WHR

#2018 data
dataWHR2018 = pd.read_csv('2018.csv') #WHR

#2019 data
dataWHR2019 = pd.read_csv('2019.csv') #WHR

In [29]:
# BEFORE WE CONTINUE WITH WORK, MUST FIND OVERLAPPING COUNTRIES FOR EVERY YEAR. WE CAN ONLY USE THOSE COUNTRIES
# MUST REMOVE COUNTRIES THAT ARE NOT IN ALL 5 YEARS or at least make them null
#ASK TA IF WE'RE ALLOWED TO ALTER THE DATA SET DIRECTLY

print("Hello World")

Hello World


In [ ]:
#hi